# 02 — Data Cleaning & Geocoding

Clean, geocode, and merge the raw datasets into analysis-ready files.

**Steps:**
1. Geocode all healthcare facilities to lat/lon
2. Validate geocoding accuracy
3. Cross-reference CMS and HRSA sources
4. Clean and join ACS demographics to census tracts
5. Merge SVI data
6. Document missing data patterns

In [ ]:
import sys
sys.path.insert(0, "..")

import geopandas as gpd
import pandas as pd

from src.config import DATA_RAW, DATA_PROCESSED
from src.geocoding import (
    geocode_facilities,
    validate_coordinates,
    cross_reference_sources,
)

## 2.1 Load Raw Data

In [ ]:
cms_path = DATA_RAW / "cms" / "cms_facilities_standardized.csv"
hrsa_path = DATA_RAW / "hrsa" / "hrsa_facilities_standardized.csv"
acs_path = DATA_RAW / "acs" / "acs_2022_tract_42.csv"
svi_path = DATA_RAW / "svi" / "SVI_2022_Pennsylvania.csv"
tracts_path = DATA_RAW / "tiger" / "tl_2024_42_tract.gpkg"

cms_df = pd.read_csv(cms_path) if cms_path.exists() else pd.DataFrame()
hrsa_df = pd.read_csv(hrsa_path) if hrsa_path.exists() else pd.DataFrame()
acs_df = pd.read_csv(acs_path)
svi_df = pd.read_csv(svi_path)
tracts_gdf = gpd.read_file(tracts_path)

cms_df.shape, hrsa_df.shape, acs_df.shape, svi_df.shape, tracts_gdf.shape

## 2.2 Geocode Healthcare Facilities

In [ ]:
cms_geo = geocode_facilities(cms_df) if not cms_df.empty else gpd.GeoDataFrame(cms_df, geometry=[], crs="EPSG:4326")
hrsa_geo = geocode_facilities(hrsa_df) if not hrsa_df.empty else gpd.GeoDataFrame(hrsa_df, geometry=[], crs="EPSG:4326")

cms_geo[[c for c in ["facility_id", "facility_name", "latitude", "longitude", "geocode_status"] if c in cms_geo.columns]].head()

## 2.3 Validate Coordinates

In [ ]:
state_boundary = tracts_gdf.dissolve().to_crs("EPSG:4326")

cms_validation = validate_coordinates(cms_geo, state_boundary) if not cms_geo.empty else pd.DataFrame()
hrsa_validation = validate_coordinates(hrsa_geo, state_boundary) if not hrsa_geo.empty else pd.DataFrame()

cms_validation.head(), hrsa_validation.head()

## 2.4 Cross-Reference and De-duplicate Facility Sources

In [ ]:
facilities_gdf = cross_reference_sources(cms_geo, hrsa_geo)
facilities_gdf.shape, facilities_gdf.head()

## 2.5 Merge Demographics and SVI to Tract Geometries

In [ ]:
tracts = tracts_gdf.copy()
tracts["geoid"] = tracts["GEOID"].astype(str)
tracts = tracts.merge(acs_df, on="geoid", how="left")

svi_df["FIPS"] = svi_df["FIPS"].astype(str).str.zfill(11)
svi_cols = [c for c in ["FIPS", "RPL_THEMES", "RPL_THEME1", "RPL_THEME2", "RPL_THEME3", "RPL_THEME4"] if c in svi_df.columns]
tracts = tracts.merge(svi_df[svi_cols], left_on="geoid", right_on="FIPS", how="left")

tracts["pct_no_vehicle"] = (tracts["households_no_vehicle"] / tracts["households"]).replace([pd.NA, pd.NaT], pd.NA) * 100
tracts["pct_uninsured"] = pd.NA
tracts["svi_overall"] = tracts.get("RPL_THEMES")

tracts_proj = tracts.to_crs(3857)
tracts["area_sq_mi"] = tracts_proj.geometry.area / 2_589_988.110336
tracts["pop_density_sq_mi"] = tracts["total_population"] / tracts["area_sq_mi"]

tracts[["geoid", "total_population", "median_household_income", "pct_no_vehicle", "svi_overall"]].head()

## 2.6 Missing Data Report

In [ ]:
missing_report = pd.DataFrame(
    {
        "column": tracts.columns,
        "missing_n": [tracts[c].isna().sum() for c in tracts.columns],
    }
)
missing_report["missing_pct"] = (missing_report["missing_n"] / len(tracts)) * 100
missing_report.sort_values("missing_pct", ascending=False).head(20)

## 2.7 Save Processed Datasets

In [ ]:
DATA_PROCESSED.mkdir(parents=True, exist_ok=True)

tracts_out = DATA_PROCESSED / "pa_tracts_enriched.gpkg"
facilities_out = DATA_PROCESSED / "pa_facilities.gpkg"
missing_out = DATA_PROCESSED / "missing_data_report.csv"

tracts.to_file(tracts_out, driver="GPKG")
facilities_gdf.to_file(facilities_out, driver="GPKG")
missing_report.to_csv(missing_out, index=False)

tracts_out, facilities_out, missing_out